In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

In [ ]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
X = df.drop('label',axis=1)

In [ ]:
y = df['label']

In [ ]:
X.shape,y.shape

((18285, 4), (18285,))

In [ ]:
messages = X.copy()

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Data Preprocessing

In [ ]:

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
  review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus

## TFidf Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(max_features=20,ngram_range=(1,3))
X=tfidf_v.fit_transform(corpus).toarray()

In [ ]:
tfidf_v.get_feature_names()[:20]

['breitbart',
 'clinton',
 'donald',
 'donald trump',
 'elect',
 'hillari',
 'hillari clinton',
 'new',
 'new york',
 'new york time',
 'obama',
 'presid',
 'say',
 'state',
 'time',
 'trump',
 'video',
 'war',
 'york',
 'york time']

In [ ]:
count_df = pd.DataFrame(X_train, columns=tfidf_v.get_feature_names())

In [ ]:
count_df.head()

,breitbart,clinton,donald,donald trump,elect,hillari,hillari clinton,new,new york,new york time,obama,presid,say,state,time,trump,video,war,york,york time
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,589
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1642
2,0,0,0,0,0,0,0,0,0,0,3024,2944,3464,3414,3534,2471,2785,4363,2670,772
3,0,0,0,0,0,0,0,0,0,0,0,0,0,2926,1241,1516,1857,2403,3642,1689
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1203,112,862,1689


##MultiNomialNB Algorithm

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB()

In [ ]:
from sklearn import metrics

In [ ]:
classifier.fit(X_train, y_train)
pred = classifier.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
metrics.confusion_matrix(y_test, pred)

accuracy:   0.729


array([[2484,  935],
       [ 699, 1917]])

In [ ]:
import tensorflow as tf


In [ ]:
tf.__version__

'2.9.2'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

##One Hot Representation

In [ ]:
voc_size = 5000

In [ ]:
onehot_repr = [one_hot(words,voc_size)for words in corpus]
onehot_repr

##Embedding Representation

In [ ]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1331 3365 1930]
 [   0    0    0 ... 1815  928 1689]
 [   0    0    0 ... 3664  658 1404]
 ...
 [   0    0    0 ... 4363 2670  772]
 [   0    0    0 ... 4259 3973 1996]
 [   0    0    0 ... 1416  304 4237]]


In [ ]:
len(embedded_docs)

18285

In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4857,
       3476, 3841, 3640,  118, 2826, 1831, 1331, 3365, 1930], dtype=int32)

##Model Creation

In [ ]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

NameError: ignored

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final,y_final,test_size=0.33, random_state=42)

##Model Training

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 45ms/step - loss: 0.3406 - accuracy: 0.8362 - val_loss: 0.1951 - val_accuracy: 0.9178
Epoch 2/10
192/192 [==============================] - 10s 52ms/step - loss: 0.1435 - accuracy: 0.9442 - val_loss: 0.1891 - val_accuracy: 0.9218
Epoch 3/10
192/192 [==============================] - 8s 44ms/step - loss: 0.1023 - accuracy: 0.9625 - val_loss: 0.2081 - val_accuracy: 0.9168
Epoch 4/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0706 - accuracy: 0.9739 - val_loss: 0.2317 - val_accuracy: 0.9157
Epoch 5/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0474 - accuracy: 0.9850 - val_loss: 0.3240 - val_accuracy: 0.9165
Epoch 6/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0305 - accuracy: 0.9895 - val_loss: 0.3411 - val_accuracy: 0.9137
Epoch 7/10
192/192 [==============================] - 8s 44ms/step - loss: 0.0197 - accuracy: 0.9946 - val_loss: 0.3938 - val_accuracy: 0.91

##Performance Metrics and Accuracy

In [ ]:
p_pred = model.predict(X_test)


189/189 [==============================] - 1s 7ms/step


In [ ]:
y_test[0:5]

array([1, 0, 0, 0, 1])

In [ ]:
p_pred[0:5]

array([[9.9975359e-01],
       [6.5377744e-06],
       [2.9377956e-03],
       [2.3007524e-06],
       [6.1427607e-05]], dtype=float32)

In [ ]:
p_pred = p_pred.flatten()
print(p_pred.round(2)[0])

1.0


In [ ]:
y_pred = np.where(p_pred > 0.5, 1, 0)
print(y_pred)

[1 0 0 ... 0 1 1]


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[3098,  321],
       [ 219, 2397]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9105219552609777

##Adding Dropout layer

In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 42ms/step - loss: 0.3418 - accuracy: 0.8373 - val_loss: 0.1965 - val_accuracy: 0.9123
Epoch 2/10
192/192 [==============================] - 8s 39ms/step - loss: 0.1536 - accuracy: 0.9411 - val_loss: 0.1870 - val_accuracy: 0.9213
Epoch 3/10
192/192 [==============================] - 8s 40ms/step - loss: 0.1121 - accuracy: 0.9580 - val_loss: 0.2149 - val_accuracy: 0.9203
Epoch 4/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0820 - accuracy: 0.9712 - val_loss: 0.2318 - val_accuracy: 0.9196
Epoch 5/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0639 - accuracy: 0.9764 - val_loss: 0.2353 - val_accuracy: 0.9185
Epoch 6/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0440 - accuracy: 0.9846 - val_loss: 0.3098 - val_accuracy: 0.9198
Epoch 7/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0312 - accuracy: 0.9904 - val_loss: 0.3313 - val_accuracy: 0.913

In [ ]:
p_pred = model.predict(X_test)
p_pred = p_pred.flatten()
print(p_pred.round(2)[0])
y_pred = np.where(p_pred > 0.5, 1, 0)
print(y_pred)

189/189 [==============================] - 2s 7ms/step
1.0
[1 0 0 ... 0 1 1]


In [ ]:
confusion_matrix(y_test,y_pred)

array([[3123,  296],
       [ 208, 2408]])

In [ ]:
accuracy_score(y_test,y_pred)*100

91.64871582435791